In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn
%matplotlib inline
plt.style.use('ggplot')
inputs = tf.placeholder(tf.float32, [None, 120, 25], 'inputs')
targets = tf.placeholder(tf.float32, [None, 25], 'targets')


/afs/inf.ed.ac.uk/user/s17/s1773005/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def get_error_and_accuracy(data):
    """Calculate average error and classification accuracy across a dataset.
    
    Args:
        data: Data provider which iterates over input-target batches in dataset.
        
    Returns:
        Tuple with first element scalar value corresponding to average error
        across all batches in dataset and second value corresponding to
        average classification accuracy across all batches in dataset.
    """
    err = 0
    acc = 0
    for input_batch, target_batch in data:
        err += sess.run(error, feed_dict={inputs: input_batch, targets: target_batch})
        acc += sess.run(accuracy, feed_dict={inputs: input_batch, targets: target_batch})
    err /= data.num_batches
    acc /= data.num_batches
    return err, acc

In [3]:
n_hidden_1 = 200
time_steps = 120
beta = 0.01 #0.009

input = tf.unstack(inputs ,time_steps,1)

weights = {
    'h1': tf.Variable(tf.random_normal([25, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, 25]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([25]))
}
def LSTM_1layer_200(data):
    lstm_layer = tf.nn.rnn_cell.GRUCell(n_hidden_1)
    LSTM_layer,_=rnn.static_rnn(lstm_layer,data,dtype="float32")
    
    out_layer = tf.matmul(LSTM_layer[-1], weights['out']) + biases['out']
    return out_layer
out_layer = LSTM_1layer_200(input)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=targets)
error = tf.reduce_mean(per_datapoint_errors)
regularizer = tf.nn.l2_loss(weights['out'])
error = tf.reduce_mean(error + beta * regularizer)

per_datapoint_pred_is_correct = tf.equal(tf.argmax(out_layer, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(error)
sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)

import data_providers_rnn as data_providers_rnn
train_data = data_providers_rnn.MSD25GenreDataProvider('train', batch_size=50, flatten=True, one_hot=True)
valid_data = data_providers_rnn.MSD25GenreDataProvider('test', batch_size=50, flatten=True, one_hot=True)

num_epoch = 50
error_train = []
error_test = []
acc_train = []
acc_test = []

for e in range(num_epoch):
    running_error = 0.
    running_acc = 0.
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [train_step, error, accuracy], 
            feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
        running_acc += batch_acc
    running_error /= train_data.num_batches
    running_acc /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error), 
          'running acc average = {1:.2f}'.format(e + 1, running_acc))
    error_train = np.append(error_train, running_error)
    acc_train = np.append(acc_train, running_acc)
    a = get_error_and_accuracy(valid_data)
    error_test = np.append(error_test, a[0])
    acc_test = np.append(acc_test, a[1])
    
Test_25 = [error_train, acc_train, error_test, acc_test]

End of epoch 1: running error average = 18.93 running acc average = 0.17
End of epoch 2: running error average = 8.25 running acc average = 0.24
End of epoch 3: running error average = 4.37 running acc average = 0.27
End of epoch 4: running error average = 3.02 running acc average = 0.29
End of epoch 5: running error average = 2.57 running acc average = 0.30
End of epoch 6: running error average = 2.40 running acc average = 0.31
End of epoch 7: running error average = 2.32 running acc average = 0.31
End of epoch 8: running error average = 2.25 running acc average = 0.33
End of epoch 9: running error average = 2.19 running acc average = 0.35
End of epoch 10: running error average = 2.14 running acc average = 0.36
End of epoch 11: running error average = 2.08 running acc average = 0.38
End of epoch 12: running error average = 2.03 running acc average = 0.40
End of epoch 13: running error average = 1.98 running acc average = 0.41
End of epoch 14: running error average = 1.94 running acc a

In [4]:
np.save('Test_25', Test_25)